In [18]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("WSDM User Logs Processing").getOrCreate() 
df_validation = spark.read.csv(
    './data/sample_submission_v2.csv', inferSchema=True, header=True
)
df_userlogs = spark.read.csv(
    './data/user_logs.csv', inferSchema=True, header=True
)
# Limit userlog 'msno' column to only include those 'msno' values present in 'df_validation', 
# so we only have the logs 
# of relevant users (in this case, the users in the validation set). 
df_validation_userlogs = df_userlogs.join(df_validation, 'msno', how='inner')

In [19]:
df_validation_userlogs.createOrReplaceTempView('user_logs')

In [20]:
spark.sql("""
SELECT * FROM user_logs
""").show()

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



KeyboardInterrupt: 

In [21]:
df = spark.sql("""
SELECT
    msno,
    avg(num_100) as avg_num_100,
    max(num_100) as max_num_100,
    min(num_100) as min_num_100,
    sum(num_100) as sum_num_100,
    stddev(num_100) as std_num_100,
    
    avg(num_25) as avg_num_25,
    max(num_25) as max_num_25,
    min(num_25) as min_num_25,
    sum(num_25) as sum_num_25,
    stddev(num_25) as std_num_25,
    
    avg(num_50) as avg_num_50,
    max(num_50) as max_num_50,
    min(num_50) as min_num_50,
    sum(num_50) as sum_num_50,
    stddev(num_50) as std_num_50,
    
    avg(num_75) as avg_num_75,
    max(num_75) as max_num_75,
    min(num_75) as min_num_75,
    sum(num_75) as sum_num_75,
    stddev(num_75) as std_num_75,
    
    avg(num_985) as avg_num_985,
    max(num_985) as max_num_985,
    min(num_985) as min_num_985,
    sum(num_985) as sum_num_985,
    stddev(num_985) as std_num_985,
    
    avg(num_unq) as avg_num_unq,
    max(num_unq) as max_num_unq,
    min(num_unq) as min_num_unq,
    sum(num_unq) as sum_num_unq,
    stddev(num_unq) as std_num_unq,
    
    avg(total_secs) as avg_total_secs,
    max(total_secs) as max_total_secs,
    min(total_secs) as min_total_secs,
    sum(total_secs) as sum_total_secs,
    stddev(total_secs) as std_total_secs
FROM user_logs
GROUP BY msno, is_churn
""").toPandas()

In [22]:
df.head()

,msno,avg_num_100,max_num_100,min_num_100,sum_num_100,std_num_100,avg_num_25,max_num_25,min_num_25,sum_num_25,...,avg_num_unq,max_num_unq,min_num_unq,sum_num_unq,std_num_unq,avg_total_secs,max_total_secs,min_total_secs,sum_total_secs,std_total_secs
0,++4RuqBw0Ss6bQU4oMxaRlbBPoWzoEiIZaxPM04Y4+U=,13.592593,128,0,5872,15.019268,7.356481,46,0,3178,...,21.379630,133,1,9236,18.703806,3985.793051,32437.844,5.340,1721862.598,4110.228254
1,+/eG3UpwFCue/nfZsF6OaTF9/aojAhLmAU9vCeY0oMw=,12.000000,39,0,204,13.057565,0.470588,3,0,8,...,8.764706,28,1,149,7.964997,3471.362294,11295.098,119.440,59013.159,3924.399063
2,+/namlXq+u3izRjHCFJV4MgqcXcLidZYszVsROOq/y4=,28.851064,417,0,4068,40.112509,24.304965,129,0,3427,...,50.304965,241,1,7093,39.579386,8149.545170,109681.417,5.878,1149085.869,10610.591422
3,+0/X9tkmyHyet9X80G6GTrDFHnJqvai8d1ZPhayT0os=,31.217852,132,0,20635,24.904138,4.753404,82,0,3142,...,29.841150,189,1,19725,24.112687,8161.983150,36909.298,0.317,5395070.862,6376.696158
4,+09YGn842g6h2EZUXe0VWeC4bBoCbDGfUboitc0vIHw=,45.048889,180,0,30408,38.858401,3.794074,79,0,2561,...,42.265185,218,1,28529,33.547113,11562.865759,44077.115,75.001,7804934.387,9925.265121


In [23]:
df.to_csv('./data/validation_ulog_features.csv', index=False)